## 2. Image Registration with Multiple and Custom Parameter Maps

### Registration

In [3]:
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk

We can run the registration with multiple stages, using the spatial transformation result from the current stage to initialize registration at the next stage.

Typically, we start with a simple spatial transformation and advance to a more complex spatial transformation.

The default, conservative, registration parameters progress from a *translation -> affine -> bspline* transformation, with 4 resolutions for each transformation type.

When multiple resolutions are used, a multi-scale image pyramid is generated with downscaled versions of the image. Registration results at a lower resolution is used to initialize registration at a higher resolution.
This improves speed and robustness.

The number of resolutions can be manually specified aswell.

A default set of registration parameters are available for

- *translation*
- *rigid*
- *affine*
- *bspline*
- *spline*
- *groupwise*

transformations. More information on these transformation and on all possible parameter settings can be found in the [elastix manual](http://elastix.isi.uu.nl/download/elastix-5.0.0-manual.pdf).

In [4]:
# Import Images
fixed_image = itk.imread('data/CT_2D_head_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_2D_head_moving.mha', itk.F)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
resolutions = 3
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid',3)
parameter_object.AddParameterMap(parameter_map_rigid)

# .. and/or load custom parameter maps from .txt file
parameter_object.AddParameterFile('data/parameters_BSpline.txt')

# ... and/or load and customize more
parameter_map_custom = parameter_object.GetDefaultParameterMap('rigid')
parameter_map_custom['Transform'] = ['BSplineTransform']
parameter_map_custom['Metric'] = ['AdvancedMattesMutualInformation']
parameter_object.AddParameterMap(parameter_map_custom)

# Save custom parameter map
parameter_object.WriteParameterFile(
    parameter_map_custom, 'parameters_custom.txt')

Registration can either be done in one line with the registration function...

In [5]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    log_to_console=False)

.. or by initiating an elastix image filter object.

In [6]:
# Load Elastix Image Filter Object
elastix_object = itk.ElastixRegistrationMethod.New()
elastix_object.SetFixedImage(fixed_image)
elastix_object.SetMovingImage(moving_image)
elastix_object.SetParameterObject(parameter_object)

# Set additional options
elastix_object.SetLogToConsole(False)

# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()

# Results of Registration
result_image = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()